In [1]:
#For more info about this code extract, look at lab10_code_extraction.ipynb
from glob import glob

In [2]:
#A list of notebooks
lab10_folders = glob('notebooks/**/lab10.ipynb', recursive=True)

In [3]:
#Number of notebooks that have saved history
len(lab10_folders)

160

In [4]:
import json

In [5]:
#Create a list of dictionaries. Each dictionary is the metadata for one of the 160 notebooks
lab10s = []
for file in lab10_folders: 
    with open(file, 'r') as f: 
        lab10s.append(json.load(f))


In [6]:
#Num files with history
len(lab10s)

160

In [8]:
#Example metadata of each cell for the 4th notebook
lab10s[3]['cells']

[{'cell_type': 'markdown',
  'metadata': {},
  'source': ['## Conditional Probability\n',
   'This lab is an introduction to visualizing conditional probabilities.  We will cover *icon arrays*.  These do not appear in the textbook and will not appear on any exam, but they will help you gain intuition about conditional probability.\n',
   '\n',
   '#### Administrative details\n',
   "This lab will not be collected.  Conditional probability will appear on the final exam, and this is an opportunity to understand it better.  **We recommend going through at least part 2.**  You can complete the rest later as an exercise when you're studying."]},
 {'cell_type': 'code',
  'execution_count': 1,
  'metadata': {'collapsed': False,
   'history': [{'code': "# Run this cell to set up the notebook, but please don't change it.\n\n# These lines import the Numpy and Datascience modules.\nimport numpy as np\nfrom datascience import *\n\n# These lines do some fancy plotting magic.\nimport matplotlib\n%ma

In [9]:
#Num cells in the 4th notebook
len(lab10s[3]['cells'])

63

In [10]:
def getCodeInCell(cell):
    """
    CELL - The metadata for one cell. 
    return - A list of code in string format.
            The number of elements in this list equals
            the number of times CELL was ran. 
            
            The number of elements in each inner list equals
            the number of lines in CELL.
            
            If the outer list doesn't contain list but just
            strings, then the number of lines of code in CELL was 1.
    """
    
    allCode = []
    
    # If CELL is a code cell and metadata isn't collapsed 
    # (there was no output from CELL), then look into metadata 
    # and extract from "metadata -> history" of CELL.
    if isCodeCell(cell) \
    and not cell['metadata']['collapsed'] \
    and 'history' in cell['metadata'] \
    and len(cell['metadata']['history']) >= 1:
        for past_code in cell['metadata']['history']:
            code = re.split(r'\n', past_code['code'])
            allCode.append(code)
                
    # If CELL is a markdown cell, extract from "source" of CELL.
    elif isMarkDownCell(cell):
        for answer_line in cell['source']:
            if answer_line == '\n':
                continue
            code = re.split(r'\n', answer_line)
            if code[len(code) - 1] == '':
                code = code[:len(code) - 1]
            allCode.append(code)
    
    # If metadata is collapsed, then extract from "source" of CELL.
    else:   
        allCode.extend(cell['source'])
    return allCode
        

In [11]:
#Returns whether CELL is a code cell.
def isCodeCell(cell):
    return cell['cell_type'] == 'code'

#Returns whether CELL is a markdown cell.
def isMarkDownCell(cell):
    return cell['cell_type'] == 'markdown'

In [12]:
import re


In [13]:
#Example metadata of a cell that was ran more than once with output.
lab10s[3]['cells'][36]

{'cell_type': 'code',
 'execution_count': 27,
 'metadata': {'collapsed': False,
  'history': [{'code': 'cancer = people.pivot("cancer status", "test status")\ncancer',
    'response': {'content': {'data': {'text/plain': 'test status | healthy | sick\nnegative    | 1       | 1\npositive    | 1       | 1'},
      'execution_count': 23,
      'metadata': {}},
     'metadata': {},
     'msg_type': 'execute_result',
     'version': '5.0'},
    'timestamp': '2016-12-01T21:51:32.777Z'},
   {'code': 'cancer = people.pivot("test status", "cancer status")\ncancer',
    'response': {'content': {'data': {'text/plain': 'cancer status | negative | positive\nhealthy       | 1        | 1\nsick          | 1        | 1'},
      'execution_count': 24,
      'metadata': {}},
     'metadata': {},
     'msg_type': 'execute_result',
     'version': '5.0'},
    'timestamp': '2016-12-01T21:51:50.813Z'},
   {'code': 'cancer = people.pivot("test status", "cancer status", "count")\ncancer',
    'response': {'cont

In [14]:
#Output from getCodeInCell given the metadata above.
getCodeInCell(lab10s[3]['cells'][36])

[['cancer = people.pivot("cancer status", "test status")', 'cancer'],
 ['cancer = people.pivot("test status", "cancer status")', 'cancer'],
 ['cancer = people.pivot("test status", "cancer status", "count")', 'cancer'],
 ['cancer = people.pivot("test status", "cancer status", "count", sum)',
  'cancer']]

In [15]:
#The regex pattern that matches the string that would 
#denote the start of a question.
Q_PATTERN = re.compile("\*\*Question ([\d+.]+)\*\*.*")

In [16]:
# An class that supports functionality to look at 
# the current head of the list. 
# Also supports the hasNext method from Java iterators.
class CellsIterator:
    def __init__(self, L):
        self.L = L
        self.i = 0
        self.n = len(L)
    def __iter__(self):
        return self
    def next(self):
        if self.i < self.n:
            result = self.L[self.i]
            self.i += 1
            return result
        else:
            raise StopIteration()
    def current(self):
        return self.L[self.i]
    def hasNext(self):
        return self.i < self.n - 1
    __next__ = next

In [17]:
# Get code in the nearest cell that is a markdown cell. 
# Intended for questions whose answers are in words. However,
# sometimes students add a code cell to display information 
# before writing their word answer.
def process_1(cells_iterator):
    while not isMarkDownCell(cells_iterator.current()):
        cells_iterator.next()
    return getCodeInCell(cells_iterator.next())

# Get code in cells in iterator until hitting a markdown cell.
def process_True(cells_iterator):
    result = []
    while not isMarkDownCell(cells_iterator.current()):
        result.append(getCodeInCell(cells_iterator.next()))
    return result

# Get code in 2nd cell in iterator (used only for question 2.1.3. and 3.2.3.)
def process_False(cells_iterator):
    result = []
    cells_iterator.next()
    result.append(getCodeInCell(cells_iterator.current()))
    return result
    
# Get code in for question 4.1. and ignore last two cells
def process_End(cells_iterator):
    result = []
    while not isMarkDownCell(cells_iterator.current()) and cells_iterator.hasNext():
        result.append(getCodeInCell(cells_iterator.next()))
    return result

In [18]:
# Process extracting the answer for question CURR_Q.
def process_q(cells_iterator, curr_q):
    if curr_q in ['2.1.3.', '3.2.3.']:
        return process_False(cells_iterator)
    elif curr_q == '3.2.4.':
        return process_1(cells_iterator)
    elif curr_q == '4.1.':
        return process_End(cells_iterator)
    else:
        return process_True(cells_iterator)
   

In [19]:
def extract_code_from_student(student_num):
    ## NOTE: Assumes that all code that pertains to a particular coding question
    # is contained in the code cells that immediately follow the markdown
    # cell that states the question.
    
    qs = {}
    curr_q = False
    cells = CellsIterator(lab10s[student_num]['cells'])
    
    while cells.hasNext():
        curr = cells.next()
        
        # If a section number has been found but not a question number, 
        # check CURR for a question number.
        if isMarkDownCell(curr) and not curr_q:
            for s in curr['source']:
                if Q_PATTERN.match(s):
                    curr_q = Q_PATTERN.search(s).group(1)
                    qs[curr_q] = []
                    break

        # If a question number has been found, extract answer.
        # If we've processed a question that is at the end of the
        # current section, then set CURR_SECTION to False so that
        # we start looking for a section number again.
        if curr_q:
            qs[curr_q] = process_q(cells, curr_q)
            curr_q = False
        
    return qs
                
            
        

In [20]:
extract_code_from_student(0)

{'2.1.': [['probability_large_green = 4/13'], [['_ = tests.grade("q21")']]],
 '2.1.1.': [['probability_green_given_large = 4/5'],
  [['_ = tests.grade("q211")']]],
 '2.1.2.': [[['# Make an icon array to help you compute the answer.',
    "displayed_grouped_icon_array(marbles.where('color', 'green').group('size'), 'green marbles')",
    '',
    '# Now compute the answer.',
    'probability_large_given_green = ...'],
   ['# Make an icon array to help you compute the answer.',
    "display_grouped_icon_array(marbles.where('color', 'green').group('size'), 'green marbles')",
    '',
    '# Now compute the answer.',
    'probability_large_given_green = ...'],
   ['# Make an icon array to help you compute the answer.',
    "display_grouped_icon_array(marbles.where('color', 'green').group('size'), 'green marbles')",
    '',
    '# Now compute the answer.',
    'probability_large_given_green = 4/10']],
  [['_ = tests.grade("q212")']]],
 '2.1.3.': [[['The first question can be answered by counti

In [21]:
questions = {'2.1.', '2.2.', '2.3.',
             '2.1.1.', '2.1.2.', '2.1.3.', 
            '3.1.1.', '3.1.2.', '3.1.3.',
            '3.2.1.', '3.2.2.', '3.2.3.', '3.2.4.', 
            '4.1.'}

In [22]:
#student #63 failed to process all questions
failed = [questions - extract_code_from_student(i).keys() for i in range(len(lab10_folders))]
[False if failed[i] == set() else i for i in range(len(failed))]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 63,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 Fa

In [23]:
#Seems that this student deleted their question 3.2.3. markdown cell
lab10s[63]['cells']

[{'cell_type': 'markdown',
  'metadata': {},
  'source': ['## Conditional Probability\n',
   'This lab is an introduction to visualizing conditional probabilities.  We will cover *icon arrays*.  These do not appear in the textbook and will not appear on any exam, but they will help you gain intuition about conditional probability.\n',
   '\n',
   '#### Administrative details\n',
   "This lab will not be collected.  Conditional probability will appear on the final exam, and this is an opportunity to understand it better.  **We recommend going through at least part 2.**  You can complete the rest later as an exercise when you're studying."]},
 {'cell_type': 'code',
  'execution_count': 1,
  'metadata': {'collapsed': False, 'history': []},
  'outputs': [{'name': 'stdout',
    'output_type': 'stream',
    'text': ['=====================================================================\n',
     'Assignment: Conditional Probability\n',
     'OK, version v1.6.4\n',
     '======================

In [24]:
correct = set(range(len(lab10_folders))) - {63}

In [25]:
lab10s_questions = {q:{i:extract_code_from_student(i)[q] for i in correct} for q in questions}

In [26]:
# A dict of dicts. Keys in outer dict are question numbers that map to another inner dict.
# Keys of inner dict are indices (that denote a student) that map to that student's answer.
lab10s_questions

{'2.1.': {0: [['probability_large_green = 4/13'],
   [['_ = tests.grade("q21")']]],
  1: [['probability_large_green = 4/13'], [['_ = tests.grade("q21")']]],
  2: [['probability_large_green = 4/13'], [['_ = tests.grade("q21")']]],
  3: [['probability_large_green = 4/13'],
   [['_ = tests.grade("q21")'], ['_ = tests.grade("q21")']]],
  4: [['probability_large_green = 4/13'], ['_ = tests.grade("q21")']],
  5: [['probability_large_green = 4/13'],
   [['_ = tests.grade("q21")'],
    ['_ = tests.grade("q21")'],
    ['_ = tests.grade("q21")']]],
  6: [['probability_large_green = 4/13'], [['_ = tests.grade("q21")']]],
  7: [['probability_large_green = 4/13'], ['_ = tests.grade("q21")']],
  8: [['probability_large_green = ...'], ['_ = tests.grade("q21")']],
  9: [['probability_large_green = 4/13'],
   [['_ = tests.grade("q21")'], ['_ = tests.grade("q21")']]],
  10: [['probability_large_green = 4/(4+1+6+2)'], ['_ = tests.grade("q21")']],
  11: [['probability_large_green = 4/13'], ['_ = tests.gra